It looks like the data label is the 'class' field:

  0 -> Hate Speech

  1 -> Offensive Language

  2 -> Neither

In [147]:
import re
from nltk.tokenize import TweetTokenizer
import pandas as pd
from google.colab import drive
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Import google drive

In [148]:
drive.mount('/content/drive')
!ls /content/drive/MyDrive/escola/nlp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
hatespeech_data.csv  hatespeech_data_tokenized.csv


# Put data into pandas

In [149]:
df = pd.read_csv("/content/drive/MyDrive/escola/nlp/hatespeech_data.csv") 

# Preprocess data

For every tweet: lowercase, remove urls, change @... and #... to 'TAG', remove 'rt', remove non-letter characters, strip leading and trailing white space. Create a list with all the words.

In [150]:
all_sentences = []
tweet_tokenizer = TweetTokenizer()

for i in range(df.shape[0]):
  tweet = ""
  s = df.iloc[i]['tweet']
  # lowercase
  s = s.lower()
  # remove urls
  s = re.sub(r'https?:\/\/\S+', ' ', s)
  s = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', s)
  # change @... and #abc to thisisatag
  s = re.sub(r'\s([@#][\w_-]+)', ' thisisatag ', s)
  # remove non-letter chars
  s = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', s)
  # thisisatag -> TAG
  s = re.sub(r" thisisatag ", ' TAG ', s)
  # remove rt
  s = re.sub(r" rt ", ' ', s)
  # remove unecessary spacing
  s = s.strip()
  s = tweet_tokenizer.tokenize(s)
  # remove punctuation after tokenization
  for j in s:
    if j not in string.punctuation:
      tweet = tweet + j + " "
  all_sentences.append(tweet[:-1]) 

# Text Dataset

In [151]:
clean_text_df = pd.DataFrame(columns=['tweet', 'label'])
for i in range(len(all_sentences)):
  # add tweet and label to clean_df
  tweet = all_sentences[i]
  label = df.iloc[i]['class']
  # is hate speech
  if label == 0:
    label = 1
  # not hate speech 
  else:
    label = 0
  clean_text_df.loc[i] = [tweet, label]

# From Text to Numbers

In [124]:
myTokenizer = Tokenizer()
myTokenizer.fit_on_texts(all_sentences)
token_sequences = myTokenizer.texts_to_sequences(all_sentences)
padded_sequences = pad_sequences(token_sequences)

# Creating data ready for training


In [143]:
clean_df = pd.DataFrame(columns=['tweet', 'label'])
for i in range(len(padded_sequences)):
  # add tweet and label to clean_df
  clean_df.loc[i] = [padded_sequences[i].tolist(), df.iloc[i]['class']]

# Exporting it as a CSV

In [153]:
clean_text_df.to_csv("/content/drive/MyDrive/escola/nlp/hatespeech_data_clean_text.csv")

In [146]:
clean_df.to_csv("/content/drive/MyDrive/escola/nlp/hatespeech_data_tokenized.csv")